This notebook shows how to use LLM-Geo, you can uncomment a study case (preferred Case 1 and Case 3) to get the spatial analysis results without any human intervention. 

Please check the [webpage of LLM-Geo](https://github.com/gladcolor/LLM-Geo) for more detail!


#  Install package

Make sure you are using the latest version of `openai` and `geopandas`.

In [1]:
# ! pip install pyvis
# ! pip install networkx
# ! pip install openai
# ! conda update  --channel conda-forge geopandas  


# Import package

In [2]:
import os
import sys
import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
from pyvis.network import Network
from openai import OpenAI
from IPython.display import display, HTML, Code
from IPython.display import clear_output

# Define Solution class
Please run the following cell to define the functions

In [3]:
%load_ext autoreload
%autoreload 2

import LLM_Geo_Constants as constants
import helper

from LLM_Geo_kernel import Solution

sys.path.append(os.path.abspath("Modules"))    
import Modules.data_eye as data_eye

# Demonstration Cases

## Input task and data desciption

In [4]:
isReview = True

In [5]:

# Case 1: population living near hazardous wastes

# task_name ='Resident_at_risk_counting'
# TASK = r"""1) Find out Census tracts that contain hazardous waste facilities, then comppute and print out the population living in those tracts. The study area is North Carolina (NC), US.
# 2) Generate a population choropleth map for all tract polygons in NC, rendering the color by tract population; and then highlight the borders of tracts that have hazardous waste facilities. Please draw all polygons, not only the highlighted ones. The map size is 15*10 inches.
# """

# data location with column information
# DATA_LOCATIONS = ["NC hazardous waste facility ESRI shape file: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/HW_Sites_EPSG4326.zip.",
#                   "NC tract boundary shapefile: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/tract_37_EPSG4326.zip. The tract ID column is 'GEOID', data types is integer.",
#                   "NC tract population CSV file: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/NC_tract_population.csv. The population is stored in 'TotalPopulation' column. The tract ID column is 'GEOID', data types is integer."
#                  ]

# data location without column information
# DATA_LOCATIONS = ["NC hazardous waste facility ESRI shape file: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/HW_Sites_EPSG4326.zip.",
#                   "NC tract boundary shapefile: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/tract_37_EPSG4326.zip.",
#                   "NC tract population CSV file: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/NC_tract_population.csv."
#                  ]
 

# # Case 3: COVID-19 death rate in US
# task_name ='COVID_death_rate'
# TASK = r'''1) Draw a choropleth map to show the death rate (death/case) of COVID-19 among the countiguous US counties. Use the accumulated COVID-19 data of 2020.12.31 to compute the death rate. Use scheme ='quantiles' when plotting the map.  Set map projection to 'Conus Albers'. Set map size to 15*10 inches.  
# 2) Draw a scatter plot to show the correlation and trend line of the death rate with the senior resident rate, including the r-square and p-value. Set data point transparency to 50%, regression line as red. Set figure size to 15*10 inches.  
# '''

# data location with column information
# DATA_LOCATIONS = [
#                   r"COVID-19 data case in 2020 (county-level): https://github.com/nytimes/covid-19-data/raw/master/us-counties-2020.csv. This data is for daily accumulated COVID cases and deaths for each county in the US. There are 5 columns: date (format: 2021-02-01), county, state, fips, cases, deaths. ",   
#                   r"Contiguous US county boundary (ESRI shapefile): https://github.com/gladcolor/spatial_data/raw/master/contiguous_counties.zip. The county FIPS column is 'GEOID'; map projection is EPSG:4269",
#                   r"Census data (ACS2020): https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county.csv. THe needed columns are: 'FIPS', 'Total Population', 'Total Population: 65 to 74 Years', 'Total Population: 75 to 84 Years', 'Total Population: 85 Years and Over'. Drop rows with NaN cells after loading the used columns.",
#                  ]

# data location without column information
# DATA_LOCATIONS = [
#                   r"COVID-19 data case in 2020 (county-level): https://github.com/nytimes/covid-19-data/raw/master/us-counties-2020.csv. This data is for daily accumulated COVID cases and deaths for each county in the US. ",   
#                   r"Contiguous US county boundary (ESRI shapefile): https://github.com/gladcolor/spatial_data/raw/master/contiguous_counties.zip.",
#                   r"Census data (ACS2020): https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county.csv. "
#                  ]

# # Case 4: Hospital_accessibility
# task_name ='Hospital_accessibility'
# TASK = r'''
# For each zipcode area in South Carolina (SC), calculate the distance from the centroid of the zipcode area to its nearest hospital, and then create a choropleth distance map of zipcode area polygons (unit: km), also show the hospital.
# '''

## data location with column information
# DATA_LOCATIONS = [
# r"SC zipcode boundary shapefile: https://github.com/GIBDUSC/test/raw/master/sc_zip_boundary.zip, the map projection is WGS1984.",
# r"SC hospitals:  https://github.com/gladcolor/spatial_data/raw/master/South_Carolina/SC_hospitals_with_emergency_room_cleaned.csv, location columns: longitude in 'Longitude' column, latitude in 'Latitude' column.",          
# ]

# data location without column information
# DATA_LOCATIONS = [
# r"SC zipcode boundary shapefile: https://github.com/GIBDUSC/test/raw/master/sc_zip_boundary.zip.",
# r"SC hospitals:  https://github.com/gladcolor/spatial_data/raw/master/South_Carolina/SC_hospitals_with_emergency_room_cleaned.csv.",          
# ]

# Case 5: Census API



# # Case 2: France_mobility_changes_2020  (NOTE: invalidated due to API shutdown)
# task_name ='France_mobility_changes_2020'
# TASK = r'''
# 1) Show the 2020 human mobility monthly change rates of each administrative regions in a France choropleth map. Each month is a sub-map in a map matrix，12 months in total. All monthly maps need to use the same colorbar range (color scheme: coolwarm). The base of the change rate is January 2020. 
# 2) Draw a line chart to show the monthly change rate trends of all administrative regeions. Each region is a line (the region name is the legend), the x-axis is 2020 months.
# '''

# DATA_LOCATIONS = ["ESRI shapefile for France administrative regions:" + \
#                   "https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip. " + \
#                   "The 'GID_1' column is the administrative region code, 'NAME_1' column is the administrative region name.",
#                   "REST API url with parameters for daily human mobility data access:" + \
#                   "http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020." + \
#                   "The response is in CSV format. There are three columns in the response: " + \
#                   "place,date (format:2020-01-07), and intra_movement. 'place' column is the administractive region code of every country;" + \
#                   "codes for France administrative regions start with 'FRA'. Use the total intra_movement of the month as the montly mobility.",
#                  ]


save_dir = os.path.join(os.getcwd(), task_name)
os.makedirs(save_dir, exist_ok=True)

# create graph
model = r'gpt-4o'
model = r'gpt-4o-2024-08-06'


## Get data overview (column names, data types, and map projection)

In [6]:
attributes_json, DATA_LOCATIONS = data_eye.add_data_overview_to_data_location(task=TASK, data_location_list=DATA_LOCATIONS, model=model)
print("DATA_LOCATIONS with data overviews:")
print(DATA_LOCATIONS)

DATA_LOCATIONS with data overviews:
['You can use the US Census Bureau API.You can use the US Census Bureau API. Data overview: ']


In [7]:
solution = Solution(
                    task=TASK,
                    task_name=task_name,
                    save_dir=save_dir,
                    data_locations=DATA_LOCATIONS,
                    model=model,
                    )
print("Prompt to get solution graph:\n")
print(solution.graph_prompt)

Prompt to get solution graph:

Your role: A professional Geo-information scientist and programmer good at Python. You have worked on Geographic information science more than 20 years, and know every detail and pitfall when processing spatial data and coding. You know well how to set up workflows for spatial analysis tasks. You have significant experence on graph theory, application, and implementation. You are also experienced on generating map using Matplotlib and GeoPandas.
 

Your task: Generate a graph (data structure) only, whose nodes are (1) a series of consecutive steps and (2) data to solve this question:  
  Show the spatial distribution of the county level median income in the contigous US. Set figure size to (25,15)
 

Your reply needs to meet these requirements: 
 1. Think step by step.
2. Steps and data (both input and output) form a graph stored in NetworkX. Disconnected components are NOT allowed.
3. Each step is a data process operation: the input can be data paths or 

## Get graph code from GPT API

In [8]:
response_for_graph = solution.get_LLM_response_for_graph() 
solution.graph_response = response_for_graph
solution.save_solution()

clear_output(wait=True)
display(Code(solution.code_for_graph, language='python'))

import networkx as nx

G = nx.DiGraph()

# 1. Load county boundary shapefile for the contiguous US
G.add_node("county_shp_path", node_type="data", data_path="path/to/contiguous_us_county_shapefile.shp", description="Path to the county boundary shapefile for the contiguous US")
G.add_node("load_county_boundary", node_type="operation", description="Load county boundary shapefile into GeoDataFrame")
G.add_edge("county_shp_path", "load_county_boundary")
G.add_node("county_boundary_gdf", node_type="data", data_path="", description="County boundary GeoDataFrame")
G.add_edge("load_county_boundary", "county_boundary_gdf")

# 2. Collect median income data through US Census Bureau API
G.add_node("median_income_api_url", node_type="data", data_path="", description="US Census Bureau API URL for county level median income")
G.add_node("collect_income_data", node_type="operation", description="Collect median income data using the Census API")
G.add_edge("median_income_api_url", "collect_income_data")
G.add_node("median_income_data", node_type="data", data_path="", description="Median income data collected")
G.add_edge("collect_income_data", "median_income_data")

# 3. Join median income data to the county boundaries
G.add_node("join_income_to_boundary", node_type="operation", description="Join median income data to county boundary GeoDataFrame on a common attribute")
G.add_edge("median_income_data", "join_income_to_boundary")
G.add_edge("county_boundary_gdf", "join_income_to_boundary")
G.add_node("county_income_gdf", node_type="data", data_path="", description="County boundary GeoDataFrame with median income data")
G.add_edge("join_income_to_boundary", "county_income_gdf")

# 4. Generate map to show spatial distribution of median income
G.add_node("generate_income_map", node_type="operation", description="Generate a map showing the spatial distribution of median income at the county level")
G.add_edge("county_income_gdf", "generate_income_map")
G.add_node("income_distribution_map", node_type="data", data_path="", description="Map showing spatial distribution of county level median income")
G.add_edge("generate_income_map", "income_distribution_map")

# Save the graph to a GraphML file
nx.write_graphml(G, "D:\\OneDrive_PSU\\OneDrive - The Pennsylvania State University\\Research_doc\\LLM_Geo\\Python_code\\Census_API\\Census_API.graphml")

## Execute code to generate the solution graph

In [9]:
exec(solution.code_for_graph)
solution_graph = solution.load_graph_file()

# Show the graph
G = nx.read_graphml(solution.graph_file)  
nt = helper.show_graph(G)
html_name = os.path.join(os.getcwd(), solution.task_name + '.html')  
# HTML file should in the same directory. See:
# https://stackoverflow.com/questions/65564916/error-displaying-pyvis-html-inside-jupyter-lab-cell
nt.show(name=html_name)
# html_name

D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM_Geo\Python_code\Census_API.html


## Generate prompts and code for operations (functions)

In [10]:
operations = solution.get_LLM_responses_for_operations(review=isReview)
solution.save_solution()

all_operation_code_str = '\n'.join([operation['operation_code'] for operation in operations])

clear_output(wait=True)
display(Code(all_operation_code_str, language='python'))

import geopandas as gpd

def load_county_boundary(county_shp_path='path/to/contiguous_us_county_shapefile.shp'):
    # Description: Load county boundary shapefile into GeoDataFrame
    county_boundary_gdf = gpd.read_file(county_shp_path)
    return county_boundary_gdf
import requests
import pandas as pd

def collect_income_data(median_income_api_url):
    # Description: Collect median income data using the Census API
    # median_income_api_url: US Census Bureau API URL for county level median income

    # Fetch the data from the API
    response = requests.get(median_income_api_url)
    response.raise_for_status()  # Raise an error for bad responses

    # Load the data into a DataFrame
    data = response.json()
    
    # Convert JSON data to DataFrame and set the first row as header
    median_income_data = pd.DataFrame(data[1:], columns=data[0])
    
    # Ensure 'GEOID' is string type to maintain leading zeros, drop rows with NaN in critical columns
    median_income_data['GEOID'] = median_income_data['GEOID'].astype(str).str.zfill(5)
    median_income_data.dropna(subset=['GEOID', 'median_income'], inplace=True)

    # Keep only necessary columns for joining
    median_income_data = median_income_data[['GEOID', 'median_income']]

    return median_income_data
import geopandas as gpd
import pandas as pd

def join_income_to_boundary(county_boundary_gdf, median_income_data):
    # Description: Join median income data to county boundary GeoDataFrame on a common attribute

    # Ensure 'GEOID' is string type to maintain leading zeros for joining
    county_boundary_gdf['GEOID'] = county_boundary_gdf['GEOID'].astype(str).str.zfill(5)

    # Drop any rows with NaN in the GEOID column in county_boundary_gdf
    county_boundary_gdf.dropna(subset=['GEOID'], inplace=True)

    # Perform the join operation
    county_income_gdf = county_boundary_gdf.merge(median_income_data, on='GEOID', how='inner')

    return county_income_gdf
import matplotlib.pyplot as plt
import geopandas as gpd

def generate_income_map(county_income_gdf):
    # Description: Generate a map showing the spatial distribution of median income at the county level
    
    # Check if county_income_gdf CRS is in a projected coordinate system, otherwise convert it
    if county_income_gdf.crs.to_string() != "EPSG:4326":
        county_income_gdf = county_income_gdf.to_crs(epsg=4326)

    # Prepare the plot
    fig, ax = plt.subplots(1, 1, figsize=(25, 15))
    
    # Plot the counties with median income
    county_income_gdf.plot(column='median_income', ax=ax, legend=True,
                           legend_kwds={'label': "Median Income by County",
                                        'orientation': "horizontal"},
                           cmap='OrRd')
    
    # Set axis off for the map
    ax.set_axis_off()
    
    # Set title
    ax.set_title('Spatial Distribution of Median Income at the County Level', 
                 fontdict={'fontsize': 20}, loc='center')

    # Save the map
    income_distribution_map = "income_distribution_map.png"
    plt.savefig(income_distribution_map, bbox_inches='tight', pad_inches=0.1)

    # Return the path to the saved map
    return income_distribution_map

## Generate prompts and code for assembly program

In [11]:
assembly_LLM_response = solution.get_LLM_assembly_response(review=isReview)
solution.assembly_LLM_response = assembly_LLM_response
solution.save_solution()

clear_output(wait=True)
display(Code(solution.code_for_assembly, language='python'))

```python
import geopandas as gpd
import pandas as pd
import requests
import matplotlib.pyplot as plt

def load_county_boundary(county_shp_path='path/to/contiguous_us_county_shapefile.shp'):
    county_boundary_gdf = gpd.read_file(county_shp_path)
    return county_boundary_gdf

def collect_income_data(median_income_api_url):
    response = requests.get(median_income_api_url)
    response.raise_for_status()

    data = response.json()
    median_income_data = pd.DataFrame(data[1:], columns=data[0])
    median_income_data['GEOID'] = median_income_data['GEOID'].astype(str).str.zfill(5)
    median_income_data.dropna(subset=['GEOID', 'median_income'], inplace=True)
    median_income_data = median_income_data[['GEOID', 'median_income']]

    return median_income_data

def join_income_to_boundary(county_boundary_gdf, median_income_data):
    county_boundary_gdf['GEOID'] = county_boundary_gdf['GEOID'].astype(str).str.zfill(5)
    county_boundary_gdf.dropna(subset=['GEOID'], inplace=True)
    

KeyboardInterrupt: 

## Execute assembly code

In [ ]:
all_code = all_operation_code_str + '\n' + solution.code_for_assembly

# display(Code(all_code, language='python'))

all_code = solution.execute_complete_program(code=all_code, try_cnt=10)